In [2]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import dash_daq as daq
import plotly.express as px
import pandas as pd
import xarray as xr
import datetime as dt
import numpy as np
import pytz
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# read all lighthouse data

lighthouse_data = {}

lighthouses = ["Narveneset", "Bohemanneset", "Daudmannsodden", "Gasoyane", "KappThordsen"]

for lighthouse in lighthouses:
    print(lighthouse)
    with xr.open_dataset(f"https://thredds.met.no/thredds/dodsC/met.no/observations/unis/lighthouse_AWS_{lighthouse}_1min") as f:
        ds = f[["temperature","wind_speed","wind_direction","relative_humidity","air_pressure"]]
        ds = ds.assign_attrs(Longitude=f.attrs["longitude"], Latitude=f.attrs["latitude"])
        ds = ds.rename({"temperature": "Temperature [°C]",
                "relative_humidity": "Relative Humidity [%]",
                "air_pressure": "Pressure [hPa]",
                "wind_speed": "Wind Speed [m/s]",
                "wind_direction": "Wind Direction [°]"})
        lighthouse_data[lighthouse] = ds



# App to check lighthouse time series for any time range
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

variables = list(lighthouse_data[list(lighthouse_data.keys())[0]].data_vars)
station_dropdown = html.Div(dcc.Dropdown(list(lighthouse_data.keys()), "Bohemanneset", id="station-dropdown", multi=True))

variable_dropdown = html.Div(dcc.Dropdown(variables, "Temperature [°C]", id="variable-dropdown"))

time_select = html.Div(dcc.DatePickerRange(id='time-range-picker',
                                            min_date_allowed=dt.date(2021, 8, 19),
                                            initial_visible_month=dt.date.today(),
                                            display_format='Y-M-D',
                                            month_format='Y-M-D',
                                            end_date_placeholder_text='Y-M-D',
                                            start_date=dt.date.today()-dt.timedelta(days=7),
                                            end_date=dt.date.today()))

utc_switch = html.Div(daq.BooleanSwitch(id='utc-switch', on=False, label="UTC", labelPosition="top"))

graph = dbc.Card(dcc.Graph(id="lighthouse-timeseries-plot"))

app.layout = dbc.Container([html.H2("Lighthouse time series", className="text-center"),
                            dbc.Row([dbc.Col(station_dropdown, width=True),
                                     dbc.Col(variable_dropdown, width=True),
                                     dbc.Col(time_select, width="auto"),
                                     dbc.Col(utc_switch, width=1)]), 
                            dbc.Row(dbc.Col(graph))], 
                            fluid=True)

@app.callback(
    Output('lighthouse-timeseries-plot', 'figure'),
    Input('time-range-picker', 'start_date'),
    Input('time-range-picker', 'end_date'),
    Input("station-dropdown", "value"),
    Input("variable-dropdown", "value"),
    Input("utc-switch", "on")
)
def update_graph(start_date, end_date, st, vari, switch_utc):
    if isinstance(st, str):
        st = [st]
    
    if vari == None:
        dr = pd.date_range(start_date, end_date, freq="1min", name="time")
        df = pd.DataFrame(np.ones((len(dr), len(st)))*np.nan, index=dr, columns=pd.Index(st, name="Station"))
        df.loc[:] = np.nan
        vari = "None"
    else:
        if len(st) == 0:
            dr = pd.date_range(start_date, end_date, freq="1min", name="time")
            df = pd.DataFrame(np.ones((len(dr)))*np.nan, index=dr, columns=pd.Index([0], name="Station"))
            df.loc[:] = np.nan
        else:
            df = [lighthouse_data[s][vari].sel(time=slice(start_date, end_date)).dropna("time").to_dataframe(name=s) for s in st]
            df = pd.concat(df, axis=1)

    

    df.index = df.index.tz_localize(tz='UTC')

    time_label = "Time [UTC]"
    if not switch_utc:
        df.index = df.index.tz_convert("Europe/Oslo")
        time_label = "Local Time"
    
    if vari == "Wind Direction [°]":
        timezone = pytz.timezone("Europe/Oslo")
        aware = timezone.localize(dt.datetime.now())
        utc_os = aware.utcoffset()
        utc_os_wd = utc_os.seconds//3600
        fig = px.scatter(df, labels={"value": vari, "time": time_label, "variable": "Station"}, range_x=[(pd.Timestamp(df.index[0]).value/1.e6)-utc_os_wd*(3600.e3), (pd.Timestamp(df.index[-1]).value/1.e6)-utc_os_wd*(3600.e3)])
        fig.update_traces(marker={'size': 3})
        fig.update_yaxes(range=[0., 360.], tickvals=[0., 45., 90., 135., 180., 225., 270., 315., 360.], ticktext=["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"],)
    else:
        fig = px.line(df, y=st, labels={"time": time_label, "variable": "Station", "value": vari})
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8052)

Narveneset
Bohemanneset
Daudmannsodden
Gasoyane
KappThordsen
